In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from LearningAlgorithms import RegressionAlgorithms
import seaborn as sns
import itertools
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
df = pd.read_pickle("../../data/processed/02_feature_data.pkl")
# --------------------------------------------------------------
# Create a training and test set
# --------------------------------------------------------------
df.columns
df_train = df.drop(["month"], axis=1)

X = df_train.drop(["Power_kW"], axis=1)
y = df_train["Power_kW"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
df.columns
basic_features = ['Board temperature_℃', 'Radiation intensity_w',
       'Wind pressure_kgm2', 'Top wind speed_MS-2', 'Low wind speed_MS-2',
       'station pressure', 'sea level pressure', 'temperature',
       'humidity', 'precipitation', 'cloud amount', 'irradiance']
pca_features = ["pca_1", "pca_2", "pca_3", "pca_4"]

print("Basic features:", len(basic_features))
print("PCA features:", len(pca_features))


feature_set_1 = list(set(basic_features))
feature_set_2 = list(set(basic_features + pca_features))

Basic features: 12
PCA features: 4


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Dense

In [ ]:
# Assuming `df` is your DataFrame
df_train = df.drop(["month"], axis=1)  # Remove 'month' column
X = df_train.drop(["Power_kW"], axis=1)  # Features
y = df_train["Power_kW"]  # Target variable

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Ensure target variable is converted to numpy arrays
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()

In [ ]:
def create_spfnet(n_layers, n_activation, kernels):
    model = tf.keras.models.Sequential()
    for i, nodes in enumerate(n_layers):
        if i == 0:
            model.add(Dense(nodes, kernel_initializer=kernels, activation=n_activation, input_dim=X_train_scaled.shape[1]))
        else:
            model.add(Dense(nodes, activation=n_activation, kernel_initializer=kernels))
    model.add(Dense(1))  # Output layer for regression
    model.compile(loss='mse', 
                  optimizer='adam',
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model

# Define the model
spfnet = create_spfnet([32, 64], 'relu', 'normal')

# Train the model
history = spfnet.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=50,
    batch_size=32,
    verbose=1
)

# Summarize the model
spfnet.summary()

c:\Users\thara\Miniconda3\envs\solar_pred\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
3140/3140 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 0.0465 - root_mean_squared_error: 0.1763 - val_loss: 4.3628e-04 - val_root_mean_squared_error: 0.0209
Epoch 2/50
3140/3140 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 3.0093e-04 - root_mean_squared_error: 0.0173 - val_loss: 6.4893e-05 - val_root_mean_squared_error: 0.0081
Epoch 3/50
3140/3140 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 1.6873e-04 - root_mean_squared_error: 0.0129 - val_loss: 6.7735e-05 - val_root_mean_squared_error: 0.0082
Epoch 4/50
3140/3140 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 1.4398e-04 - root_mean_squared_error: 0.0119 - val_loss: 2.0087e-04 - val_root_mean_squared_error: 0.0142
Epoch 5/50
3140/3140 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 1.3362e-04 - root_mean_squared_error: 0.0114 - val_loss: 9.1228e-05 - val_root_mean_squared_error: 0.0096
Epoch 6/50
3140/3140 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 1.0850e-04 - root_mean_squared_error: 0.0103 - val_loss: 2.1180e-04 - val_root_mean_squared_error: 0.0

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,165 (31.90 KB)

 Trainable params: 2,721 (10.63 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,444 (21.27 KB)